Reference: https://github.com/dh1105/Sentence-Entailment

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install transformers
# !pip install sentencepiece
## !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

In [3]:
import pandas as pd
import re
import torch
# import torch_xla
# import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
# from keras.preprocessing.sequence import pad_sequences
import pickle
import os
import numpy as np

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
task_a_hyp = "This is offensive speech."
task_b_hyp = "This is targeted offense."
task_c_hyp_1= "This is targeted towards an individual."
task_c_hyp_2 = "This is targeted towards a group."
task_c_hyp_3 = "This isn't targeted towards a group or an individual."
# f = open('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Data/olid_train_v2.csv')
# f_a = open('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Data/bert_nli.csv', 'w')
f = open('Data/olid_train_v2.csv')
f_a = open('Data/bert_nli.csv', 'w')
f_a.write('tweet_id' + '\t' + 'gold_label' + '\t' + 'sentence1' + '\t' + 'sentence2' + '\n')
lines = f.readlines()
for line in lines[1:]:
  row = line.split('\t')
  tweet_id = row[0].strip()
  tweet_text = ' '.join(row[1:-3]).strip()
  is_offensive = row[-3].strip()
  is_targeted = row[-2].strip()
  target = row[-1].strip()
  if is_offensive == 'OFF':
    f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
    if is_targeted == 'TIN':
      f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
      if target == 'IND':
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_3 + '\n')
      elif target == 'GRP':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_3 + '\n')
      elif target == 'OTH':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp_3 + '\n')
    elif is_targeted == 'UNT':
      f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
  else:
    f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')

In [6]:
#df = pd.read_csv('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Data/bert_nli.csv', delimiter='\t')
df = pd.read_csv('./Data/bert_nli.csv', delimiter='\t')

In [7]:
df = df.sample(frac=1)

In [8]:
l = len(df)
train_df = df[:int(0.8*l)]
val_df = df[int(0.8*l):int(0.9*l)]
test_df = df[int(0.9*l):]

In [9]:
train_df = train_df.dropna()
val_df = val_df.dropna()
test_df = test_df.dropna()

In [10]:
train_df['sentence1'] = train_df['sentence1'].astype(str)
train_df['sentence2'] = train_df['sentence2'].astype(str)

In [11]:
val_df['sentence1'] = val_df['sentence1'].astype(str)
val_df['sentence2'] = val_df['sentence2'].astype(str)

In [12]:
test_df['sentence1'] = test_df['sentence1'].astype(str)
test_df['sentence2'] = test_df['sentence2'].astype(str)

In [13]:
train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]
val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]
test_df = test_df[(test_df['sentence1'].str.split().str.len() > 0) & (test_df['sentence2'].str.split().str.len() > 0)]

In [14]:
train_df

,tweet_id,gold_label,sentence1,sentence2
6264,18807,contradiction,@USER Also Puerto Rico is a island add in the ...,This is offensive speech.
21153,31019,contradiction,@USER @USER @USER Fuck you... If she dare the...,This is targeted towards a group.
21449,87056,entailment,@USER @USER No they aren’t! They have all said...,This is targeted towards a group.
27957,48800,contradiction,1/2 URL So because he speaks in a manner that ...,This is offensive speech.
277,59998,entailment,@USER You can probably imagine all the SJW sno...,This is targeted offense.
...,...,...,...,...
25116,87588,contradiction,This explains a lot: URL #MAGA,This is offensive speech.
22871,13519,contradiction,@USER @USER @USER @USER Don't you liberals wan...,This is offensive speech.
26855,36284,contradiction,@USER We get shit transported from Africa nigg...,This is targeted towards an individual.
17086,95833,contradiction,@USER @USER @USER Wow! I like this lady more ...,This is offensive speech.


In [15]:
val_df

,tweet_id,gold_label,sentence1,sentence2
14065,97581,entailment,@USER you suck so fucking hard that i genuinel...,This is targeted offense.
23666,60683,entailment,@USER Wake up black Americans! #maga #Kavanau...,This is targeted towards a group.
17606,12476,contradiction,@USER @USER @USER Trump is declassifying infor...,This is offensive speech.
12118,39490,contradiction,@USER @USER @USER Whenever conservatives lose ...,This isn't targeted towards a group or an indi...
4916,55559,contradiction,#TuesdayThought. The only hate speech that I...,This is targeted towards a group.
...,...,...,...,...
4381,89799,entailment,Impeach Trump and #MAGA,This is offensive speech.
17250,49204,contradiction,@USER AARON RODGERS WILL NEVER WIN A RING THE ...,This is targeted towards a group.
26653,96023,contradiction,@USER @USER “has been the inspiration for the ...,This is offensive speech.
5526,59858,entailment,@USER @USER @USER @USER Chris Burns you are a ...,This is targeted offense.


In [16]:
test_df

,tweet_id,gold_label,sentence1,sentence2
3092,77105,entailment,@USER in the fake national emergency that was ...,This is offensive speech.
23144,46226,contradiction,@USER Who the hell does @USER think he is to s...,This is offensive speech.
5560,17595,entailment,@USER @USER @USER No. She is a liar.,This is targeted towards an individual.
5372,34887,entailment,@USER @USER yeah keep mocking her right into a...,This is targeted offense.
10122,23596,contradiction,@USER Fear about losing your monopoly status i...,This is offensive speech.
...,...,...,...,...
1763,96139,entailment,@USER You are TRIPPIN. Stop jumping to conclus...,This is targeted offense.
13683,90804,contradiction,@USER @USER HIRE AMERICAN YOU PIECES OF TRASH!...,This isn't targeted towards a group or an indi...
2648,30412,entailment,@USER How he is still batting,This is targeted offense.
255,52824,contradiction,@USER Good girl @USER,This is offensive speech.


In [17]:
test_df_a = test_df[test_df.sentence2 == "This is offensive speech."]
test_df_b = test_df[test_df.sentence2 == "This is targeted offense."]
test_df_c = test_df[test_df.sentence2 == "This is targeted towards an individual."]
test_df_d = test_df[test_df.sentence2 == "This is targeted towards a group."]
test_df_e = test_df[test_df.sentence2 == "This isn't targeted towards a group or an individual."]

In [18]:
print(len(test_df_a))
print(len(test_df_b))
print(len(test_df_c))
print(len(test_df_d))
print(len(test_df_e))

1271
436
364
362
383


In [19]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pickle
import os
from transformers import BertTokenizer

class MNLIDataBert(Dataset):

  def __init__(self, train_df, val_df, test_df, test_df_a, test_df_b, test_df_c, test_df_d, test_df_e):
    self.label_dict = {'entailment': 0, 'contradiction': 1}

    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df
    self.test_df_a = test_df_a
    self.test_df_b = test_df_b
    self.test_df_c = test_df_c
    self.test_df_d = test_df_d
    self.test_df_e = test_df_e
    #self.base_path = '/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/mnli-data'
    self.base_path = 'mnli-data'
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    self.train_data = None
    self.val_data = None
    self.test_data = None
    self.test_data_a = None
    self.test_data_b = None
    self.test_data_c = None
    self.test_data_d = None
    self.test_data_e = None
    self.init_data()

  def init_data(self):
    #Saving takes too much RAM
    
    if os.path.exists(os.path.join(self.base_path, 'train_data.pkl')):
      print("Found training data")
      with open(os.path.join(self.base_path, 'train_data.pkl'), 'rb') as f:
        self.train_data = pickle.load(f)
    else:
      self.train_data = self.load_data(self.train_df)
      with open(os.path.join(self.base_path, 'train_data.pkl'), 'wb') as f:
        pickle.dump(self.train_data, f)
    if os.path.exists(os.path.join(self.base_path, 'val_data.pkl')):
      print("Found val data")
      with open(os.path.join(self.base_path, 'val_data.pkl'), 'rb') as f:
        self.val_data = pickle.load(f)
    else:
      self.val_data = self.load_data(self.val_df)
      with open(os.path.join(self.base_path, 'val_data.pkl'), 'wb') as f:
        pickle.dump(self.val_data, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data.pkl')):
      print("Found test data")
      with open(os.path.join(self.base_path, 'test_data.pkl'), 'rb') as f:
        self.test_data = pickle.load(f)
    else:
      self.test_data = self.load_data(self.test_df)
      with open(os.path.join(self.base_path, 'test_data.pkl'), 'wb') as f:
        pickle.dump(self.test_data, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_a.pkl')):
      print("Found test data a")
      with open(os.path.join(self.base_path, 'test_data_a.pkl'), 'rb') as f:
        self.test_data_a = pickle.load(f)
    else:
      self.test_data_a = self.load_data(self.test_df_a)
      with open(os.path.join(self.base_path, 'test_data_a.pkl'), 'wb') as f:
        pickle.dump(self.test_data_a, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_b.pkl')):
      print("Found test data b")
      with open(os.path.join(self.base_path, 'test_data_b.pkl'), 'rb') as f:
        self.test_data_b = pickle.load(f)
    else:
      self.test_data_b = self.load_data(self.test_df_b)
      with open(os.path.join(self.base_path, 'test_data_b.pkl'), 'wb') as f:
        pickle.dump(self.test_data_b, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_c.pkl')):
      print("Found test data c")
      with open(os.path.join(self.base_path, 'test_data_c.pkl'), 'rb') as f:
        self.test_data_c = pickle.load(f)
    else:
      self.test_data_c = self.load_data(self.test_df_c)
      with open(os.path.join(self.base_path, 'test_data_c.pkl'), 'wb') as f:
        pickle.dump(self.test_data_c, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_d.pkl')):
      print("Found test data d")
      with open(os.path.join(self.base_path, 'test_data_d.pkl'), 'rb') as f:
        self.test_data_d = pickle.load(f)
    else:
      self.test_data_d = self.load_data(self.test_df_d)
      with open(os.path.join(self.base_path, 'test_data_d.pkl'), 'wb') as f:
        pickle.dump(self.test_data_d, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_e.pkl')):
      print("Found test data e")
      with open(os.path.join(self.base_path, 'test_data_e.pkl'), 'rb') as f:
        self.test_data_e = pickle.load(f)
    else:
      self.test_data_e = self.load_data(self.test_df_e)
      with open(os.path.join(self.base_path, 'test_data_e.pkl'), 'wb') as f:
        pickle.dump(self.test_data_e, f)
    # self.train_data = self.load_data(self.train_df)
    # self.val_data = self.load_data(self.val_df)
    # self.test_data = self.load_data(self.test_df)
  def load_data(self, df):
    MAX_LEN = 512
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['sentence1'].to_list()
    hypothesis_list = df['sentence2'].to_list()
    label_list = df['gold_label'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    print(len(dataset))
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader = DataLoader(
      self.test_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_a = DataLoader(
      self.test_data_a,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_b = DataLoader(
      self.test_data_b,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_c = DataLoader(
      self.test_data_c,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_d = DataLoader(
      self.test_data_d,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_e = DataLoader(
      self.test_data_e,
      shuffle=shuffle,
      batch_size=batch_size
    )

    return train_loader, val_loader, test_loader, test_loader_a, test_loader_b, test_loader_c, test_loader_d, test_loader_e

In [20]:
mnli_dataset = MNLIDataBert(train_df, val_df, test_df, test_df_a, test_df_b, test_df_c, test_df_d, test_df_e)

Found training data
Found val data
Found test data
Found test data a
Found test data b
Found test data c
Found test data d
Found test data e


In [21]:
train_loader, val_loader, test_loader, test_loader_a, test_loader_b, test_loader_c, test_loader_d, test_loader_e = mnli_dataset.get_data_loaders(batch_size=64)

In [22]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [23]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [24]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False)

In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 109,483,778 trainable parameters


In [26]:
def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [27]:
import time

EPOCHS = 5

def train(model, train_loader, val_loader, optimizer):  
  total_step = len(train_loader)
  #model.load_state_dict(torch.load('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Weights/nli_0.pt'))
  #model.load_state_dict(torch.load('./Weights/nli_3.pt'))
  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(train_loader):
      optimizer.zero_grad()
      print(batch_idx, len(train_loader))
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()

      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(val_loader):
        optimizer.zero_grad()
        pair_token_ids = pair_token_ids.to(device)
        mask_ids = mask_ids.to(device)
        seg_ids = seg_ids.to(device)
        labels = y.to(device)

        # prediction = model(pair_token_ids, mask_ids, seg_ids)
        loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()
        
        # loss = criterion(prediction, labels)
        acc = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    #torch.save(model.state_dict(),os.path.join('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Weights', 'nli_{}.pt'.format(epoch)))
    torch.save(model.state_dict(),os.path.join('./Weights', 'nli_new_{}.pt'.format(epoch)))
    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [30]:
import time
def test(model, test_loader):
  model.load_state_dict(torch.load('Weights/nli_new_4.pt', map_location=torch.device('cpu')))
  model.eval()
  total_test_acc  = 0
  total_test_loss = 0
  start = time.time()
  preds = []
  acts = []
  with torch.no_grad():
    #print("Prediction", "Label")
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(test_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)

      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                           token_type_ids=seg_ids, 
                           attention_mask=mask_ids, 
                           labels=labels).values()
        
      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)
      preds = preds + prediction.tolist()
      acts = acts + labels.tolist()
      #print(prediction, labels)
      total_test_loss += loss.item()
      total_test_acc  += acc.item() 

  test_acc  = total_test_acc/len(test_loader)
  test_loss = total_test_loss/len(test_loader)
  end = time.time()
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)
  print(f'test_loss: {test_loss:.4f} test_acc: {test_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
  return [0 if preds[i][0] > preds[i][1] else 1 for i in range(len(preds))], acts

In [28]:
train(model, train_loader, val_loader, optimizer)

0 366
1 366
2 366
3 366
4 366
5 366
6 366
7 366
8 366
9 366
10 366
11 366
12 366
13 366
14 366
15 366
16 366
17 366
18 366
19 366
20 366
21 366
22 366
23 366
24 366
25 366
26 366
27 366
28 366
29 366
30 366
31 366
32 366
33 366
34 366
35 366
36 366
37 366
38 366
39 366
40 366
41 366
42 366
43 366
44 366
45 366
46 366
47 366
48 366
49 366
50 366
51 366
52 366
53 366
54 366
55 366
56 366
57 366
58 366
59 366
60 366
61 366
62 366
63 366
64 366
65 366
66 366
67 366
68 366
69 366
70 366
71 366
72 366
73 366
74 366
75 366
76 366
77 366
78 366
79 366
80 366
81 366
82 366
83 366
84 366
85 366
86 366
87 366
88 366
89 366
90 366
91 366
92 366
93 366
94 366
95 366
96 366
97 366
98 366
99 366
100 366
101 366
102 366
103 366
104 366
105 366
106 366
107 366
108 366
109 366
110 366
111 366
112 366
113 366
114 366
115 366
116 366
117 366
118 366
119 366
120 366
121 366
122 366
123 366
124 366
125 366
126 366
127 366
128 366
129 366
130 366
131 366
132 366
133 366
134 366
135 366
136 366
137 366
138 36

In [31]:
preds, acts = test(model, test_loader)

test_loss: 0.6206 test_acc: 0.8349
00:00:26.37


In [32]:
acc = 0
for i in range(len(preds)):
    acc += 1 if preds[i] == acts[i] else 0
print("Overall test accuracy is", acc/len(acts))

Overall test accuracy is 0.8343013324222753


In [33]:
preds_a, acts_a = test(model, test_loader_a)

test_loss: 0.8757 test_acc: 0.7802
00:00:09.94


In [34]:
ent_acts_a = 0
cont_acts_a = 0
correct_ents_a = 0
correct_conts_a = 0
for i in range(len(preds_a)):
    if acts_a[i] == 0:
        ent_acts_a += 1
        if preds_a[i] == 0:
            correct_ents_a += 1
    else:
        cont_acts_a += 1
        if preds_a[i] == 1:
            correct_conts_a += 1
            
print("Total offensive examples:", ent_acts_a)
print("Correctly predicted offensive examples:", correct_ents_a)
print("Accuracy for predicting offensive examples:", correct_ents_a/ent_acts_a)
print("Total non offensive examples:", cont_acts_a)
print("Correctly predicted non offensive examples:", correct_conts_a)
print("Accuracy for predicting non offensive examples:", correct_conts_a/cont_acts_a)

Total offensive examples: 414
Correctly predicted offensive examples: 281
Accuracy for predicting offensive examples: 0.678743961352657
Total non offensive examples: 897
Correctly predicted non offensive examples: 742
Accuracy for predicting non offensive examples: 0.8272017837235228


In [35]:
preds_b, acts_b = test(model, test_loader_b)

test_loss: 0.6044 test_acc: 0.8610
00:00:02.78


In [36]:
ent_acts_b = 0
cont_acts_b = 0
correct_ents_b = 0
correct_conts_b = 0
for i in range(len(preds_b)):
    if acts_b[i] == 0:
        ent_acts_b += 1
        if preds_b[i] == 0:
            correct_ents_b += 1
    else:
        cont_acts_b += 1
        if preds_b[i] == 1:
            correct_conts_b += 1
            
print("Total targeted offensive examples:", ent_acts_b)
print("Correctly predicted targeted offensive examples:", correct_ents_b)
print("Accuracy for predicting targeted offensive examples:", correct_ents_b/ent_acts_b)
print("Total non targeted offensive examples:", cont_acts_b)
print("Correctly predicted non targeted offensive examples:", correct_conts_b)
print("Accuracy for predicting non targeted offensive examples:", correct_conts_b/cont_acts_b)

Total targeted offensive examples: 408
Correctly predicted targeted offensive examples: 402
Accuracy for predicting targeted offensive examples: 0.9852941176470589
Total non targeted offensive examples: 61
Correctly predicted non targeted offensive examples: 4
Accuracy for predicting non targeted offensive examples: 0.06557377049180328


In [37]:
len(train_df[(train_df.sentence2 == "This is targeted offense.") & (train_df.gold_label == 'contradiction')])

391

In [38]:
len(train_df[(train_df.sentence2 == "This is targeted offense.") & (train_df.gold_label == 'entailment')])

2967

In [39]:
'''The above two cells is one possible explanation of why the model always predicts entailment for task_b'''

'The above two cells is one possible explanation of why the model always predicts entailment for task_b'

In [40]:
preds_c, acts_c = test(model, test_loader_c)

test_loss: 0.5066 test_acc: 0.8659
00:00:02.52


In [41]:
ent_acts_c = 0
cont_acts_c = 0
correct_ents_c = 0
correct_conts_c = 0
for i in range(len(preds_c)):
    if acts_c[i] == 0:
        ent_acts_c += 1
        if preds_c[i] == 0:
            correct_ents_c += 1
    else:
        cont_acts_c += 1
        if preds_c[i] == 1:
            correct_conts_c += 1
            
print("Total individual targeted offensive examples:", ent_acts_c)
print("Correctly predicted individual targeted offensive examples:", correct_ents_c)
print("Accuracy for predicting individual targeted offensive examples:", correct_ents_c/ent_acts_c)
print("Total non individual targeted offensive examples:", cont_acts_c)
print("Correctly predicted non individual targeted offensive examples:", correct_conts_c)
print("Accuracy for predicting non individual targeted offensive examples:", correct_conts_c/cont_acts_c)

Total individual targeted offensive examples: 224
Correctly predicted individual targeted offensive examples: 198
Accuracy for predicting individual targeted offensive examples: 0.8839285714285714
Total non individual targeted offensive examples: 135
Correctly predicted non individual targeted offensive examples: 114
Accuracy for predicting non individual targeted offensive examples: 0.8444444444444444


In [42]:
preds_d, acts_d = test(model, test_loader_d)

test_loss: 0.3581 test_acc: 0.8594
00:00:02.64


In [43]:
ent_acts_d = 0
cont_acts_d = 0
correct_ents_d = 0
correct_conts_d = 0
for i in range(len(preds_d)):
    if acts_d[i] == 0:
        ent_acts_d += 1
        if preds_d[i] == 0:
            correct_ents_d += 1
    else:
        cont_acts_d += 1
        if preds_d[i] == 1:
            correct_conts_d += 1
            
print("Total group targeted offensive examples:", ent_acts_d)
print("Correctly predicted group targeted offensive examples:", correct_ents_d)
print("Accuracy for predicting group targeted offensive examples:", correct_ents_d/ent_acts_d)
print("Total non group targeted offensive examples:", cont_acts_d)
print("Correctly predicted non group targeted offensive examples:", correct_conts_d)
print("Accuracy for predicting non group targeted offensive examples:", correct_conts_d/cont_acts_d)

Total group targeted offensive examples: 116
Correctly predicted group targeted offensive examples: 94
Accuracy for predicting group targeted offensive examples: 0.8103448275862069
Total non group targeted offensive examples: 276
Correctly predicted non group targeted offensive examples: 249
Accuracy for predicting non group targeted offensive examples: 0.9021739130434783


In [44]:
preds_e, acts_e = test(model, test_loader_e)

test_loss: 0.2499 test_acc: 0.9055
00:00:03.58


In [45]:
ent_acts_e = 0
cont_acts_e = 0
correct_ents_e = 0
correct_conts_e = 0
for i in range(len(preds_e)):
    if acts_e[i] == 0:
        ent_acts_e += 1
        if preds_e[i] == 0:
            correct_ents_e += 1
    else:
        cont_acts_e += 1
        if preds_e[i] == 1:
            correct_conts_e += 1
            
print("Total other targeted offensive examples:", ent_acts_e)
print("Correctly predicted other targeted offensive examples:", correct_ents_e)
print("Accuracy for predicting other targeted offensive examples:", correct_ents_e/ent_acts_e)
print("Total non other targeted offensive examples:", cont_acts_e)
print("Correctly predicted non other targeted offensive examples:", correct_conts_e)
print("Accuracy for predicting non other targeted offensive examples:", correct_conts_e/cont_acts_e)

Total other targeted offensive examples: 46
Correctly predicted other targeted offensive examples: 9
Accuracy for predicting other targeted offensive examples: 0.1956521739130435
Total non other targeted offensive examples: 350
Correctly predicted non other targeted offensive examples: 349
Accuracy for predicting non other targeted offensive examples: 0.9971428571428571


In [46]:
len(train_df[(train_df.sentence2 == "This isn't targeted towards a group or an individual.") & (train_df.gold_label == 'entailment')])

302

In [47]:
len(train_df[(train_df.sentence2 == "This isn't targeted towards a group or an individual.") & (train_df.gold_label == 'contradiction')])

2678

In [48]:
'''The above two examples give us an idea as to why the model might not be predicting any positive other targeted examples'''

'The above two examples give us an idea as to why the model might not be predicting any positive other targeted examples'